In [ ]:
# Simplifying Data Management Using the Cityworks API and Python
# June 2021
#
# Miklos Nadas, GISP
# miklos_nadas@clevelandwater.com 
#
# City of Cleveland

# Agenda

# 1. Project Description - Bulk Importing Inspections into Cityworks
#    a. Background
#        i. Survey123 was used to inventory and inspect Guardrails for the Department of Streets
#       ii. Cityworks should contain all work history information
#    b. Problem Statement - How to import ~3000 guardrail inspections from Survey123 to Cityworks
#    c. Solution - Python & Cityworks API
#
# 2. Introduction
#    a. Python & Jupyter
#    b. Miklos Nadas, GISP
#    c. City of Cleveland 
#
# 3. Cityworks at the City of Cleveland
#    a. Number of Users
#    b. Number of the types of Work Activities
#    c. Amount of Work Activities
#
# 4. Cityworks API
#    a. Documentation
#    b. Download
#    c. Install
#    d. Number of Cityworks Users
#    e. Number of the types of Work Activities
#
# 5. Cityworks API and Python
#    a. Manually Create Inspection
#    b. Dissect Inspection creation into smaller API components
#    c. Combine Everything to Bulk Upload Inspections
#
# 6. Closing Thoughts and Moving forward

In [ ]:
# The libraries used for this presentation

from arcgis import GIS             # Mapping, spatial analysis, data science, geospatial AI and automation
import pwd                         # Personal password database
import pypyodbc                    # Connect to SQL Server
from IPython.display import Image  # Displays images in Jupyter
import webbrowser                  # Web-browser controller
import pandas                      # Data analysis and manipulation tool
import os                          # Miscellaneous operating system interfaces

In [ ]:
# My work history

url = r'https://www.linkedin.com/in/miklosnadas'
webbrowser.open(url)

In [ ]:
# City of Cleveland logo

city_image = r'C:\data\Cityworks_Innovate_2021\pics\City-Logo.jpg'
Image(filename=city_image, width=200)

In [ ]:
# Connecting to ArcGIS Online

gis = GIS(r'http://www.arcgis.com/', 'user', pwd.getagolpwd())

In [ ]:
# Creating a Map to show the location of Cleveland, OH

map_cleveland = gis.map('Ohio')
# Cleveland Point
cleve_pnt = 'abcde12345'

cleve_pnt_fs = gis.content.get(cleve_pnt)
map_cleveland.add_layer(cleve_pnt_fs)

In [ ]:
map_cleveland

In [ ]:
# Adding the Cleveland Boundary

cleve_boundary = 'bcdef234567'
cleve_boundary_fs = gis.content.get(cleve_boundary)

map_cleveland.add_layer(cleve_boundary_fs.layers[0])
map_cleveland.extent = [[-81.6, 41.7], [-81.8, 41.3]]

In [ ]:
# Population of Cleveland using USA Block Groups
block_groups = 'cdefgh345678'

block_fs = gis.content.get(block_groups)

sdf = pandas.DataFrame.spatial.from_layer(block_fs.layers[0])

In [ ]:
sdf.head()

In [ ]:
# Population of Cleveland

print("2012 Population Cleveland from Census Block Groups: {}".format('{:,}'.format(sdf.loc[sdf['WITHINCOC'] == 'Yes', 'POP2012'].sum())))

In [ ]:
# Connection to Cityworks Database

f = open(r"C:\data\Cityworks_Innovate_2021\sql_conn.txt", "r")
sqlconn = f.read()

server = 'Server={}; '.format(sqlconn)

f = open(r"C:\data\Cityworks_Innovate_2021\sql_user.txt", "r")
sqluser = f.read()

connection = pypyodbc.connect('Driver={SQL Server}; ' + server + 
                              'Database=cityworksdatabase; '
                             r'uid=' + sqluser + ';pwd=' + pwd.getpythonagolpwd())

cursor = connection.cursor()

In [ ]:
# Current City of Cleveland Domains

domainSQLcommand = ("SELECT [DOMAINNAME] "
                    ",[DESCRIPTION]"
                    "FROM [cityworksdatabase].[azteca].[CWDOMAIN]")

cursor.execute(domainSQLcommand)

i = 0
for row in cursor.fetchall():
    print("Domain ID:\t{}\t\tDomain Description:\t{}".format(row[0], row[1]))
    i += 1
    
print("\nNumber of Domains: {}".format(str(i)))

In [ ]:
# Current Number of Active Cityworks users

domainSQLcommand = ("SELECT count(EMPLOYEESID) "                    
                    "FROM [cityworksdatabase].[azteca].[EMPLOYEE]"
                    "where EMPLOYEE.ISACTIVE = 'Y'")

cursor.execute(domainSQLcommand)

for row in cursor.fetchall():
    print("Number of active Cityworks Users: {}".format(str(row[0])))

In [ ]:
# Number of WO templates, Service Request Templates, and Inspection Templates

domainSQLcommand = ("SELECT count(WOTEMPLATEID) "                    
                    "FROM [cityworksdatabase].[azteca].[WOTEMPLATE]"
                   # "where DOMAINID=2"
                   )

cursor.execute(domainSQLcommand)

for row in cursor.fetchall():
    wotemplates = row[0]
    
domainSQLcommand = ("SELECT count(INSPTEMPLATEID) "                    
                    "FROM [cityworksdatabase].[azteca].[INSPTEMPLATE]"
                   # "where DOMAINID=2"
                   )

cursor.execute(domainSQLcommand)

for row in cursor.fetchall():
    insptemplates = row[0]
    
domainSQLcommand = ("SELECT count(PROBLEMSID) "                    
                    "FROM [cityworksdatabase].[azteca].[PROBLEMLEAF]"
                   # "where DOMAINID=2"
                   )

cursor.execute(domainSQLcommand)

for row in cursor.fetchall():
    srtemplates = row[0]
    
print("Number of Service Request Templates: {}".format(str(srtemplates)))
print("Number of Inspection Templates: {}".format(str(insptemplates)))
print("Number of Work Order Templates: {}".format(str(wotemplates)))

In [ ]:
# Number of Work Orders, Service Requests, and Inspections

domainSQLcommand = ("SELECT count([REQUESTID]) "
                    ",MIN(DATETIMEINIT)"
                    "FROM [cityworksdatabase].[azteca].[REQUEST]")

cursor.execute(domainSQLcommand)

for row in cursor.fetchall():
    srnum, srdate = row
    
domainSQLcommand = ("SELECT count([WORKORDERID]) "
                    ",MIN(INITIATEDATE)"
                    "FROM [cityworksdatabase].[azteca].[WORKORDER]")

cursor.execute(domainSQLcommand)

for row in cursor.fetchall():
    wonum, wodate = row
    
domainSQLcommand = ("SELECT count([INSPECTIONID]) "
                    ",MIN(INSPDATE)"
                    "FROM [cityworksdatabase].[azteca].[INSPECTION]")

cursor.execute(domainSQLcommand)

for row in cursor.fetchall():
    inspnum, inspdate = row
    
print("Earliest Work Activity Year:\t{}\n".format(str(srdate.year)))
    
print("Number of Service Requests:\t{}".format('{:,}'.format(srnum)))
print("Number of Inspections:\t\t{}".format('{:,}'.format(inspnum)))
print("Number of Work Orders:\t\t{}".format('{:,}'.format(wonum)))

In [ ]:
# All Cityworks instances install the API documentation

url = r'https://www.cityworksurl.com/cityworks/apidocs/'
webbrowser.open(url)

In [ ]:
# Download the SDK for your current Cityworks version

url = r'https://mycityworks.force.com/s/article/Cityworks15-6-4Download'
webbrowser.open(url)

In [ ]:
# Installing Cityworks API on your Python Instance
# in Command Prompt
# <Python install location> <Cityworks Python API Install> install

# C:\Python\python.exe C:\Data\temp\python\setup.py install

In [ ]:
# Cityworks libraries used to create API calls

import cwpy.cwServices, cwpy.cwMessagesAMS # to get the Cityworks token
import requests, json # to make the rest of the Cityworks API Calls

In [ ]:
# Function to convert Python dictionary to JSON.

def data_to_json(data_dict): 
    token = cw_token
    json_data = json.dumps(data_dict, separators=(",",":"))
    if len(list(token)) == 0:
        params = {"data": json_data}
    else:
        params = {"token": token, "data": json_data}
    return params

In [ ]:
# Function to make an API call.

def make_request(url, params):  
    response = requests.get(url, params=params)
    return json.loads(response.text)

In [ ]:
# City of Cleveland Test Instance

base_url = "https://www.cityworksurl.com/cityworks"

In [ ]:
# To get the Cityworks Token

services = cwpy.cwServices.Services()
services.url = base_url
base_url = base_url + r'/Services/'
username = "mnadas"
password = pwd.getcwpwd()
auth_response = services.authenticate(username, password)
cw_token = auth_response["Value"]["Token"]

In [ ]:
# Number of Employees

url = base_url + 'Ams/Employee/All'
data = {"IncludeInactive": 0}
parameters = data_to_json(data)
all_employees = make_request(url, parameters)

print("Number of active Cityworks Users: {}\n".format(str(len(all_employees['Value']))))

exampleURL = r'https://www.cityworksurl.com/cityworks/Services/Ams/Employee/All?data={"IncludeInactive":0}&token=' + cw_token
print(exampleURL)
# webbrowser.open(exampleURL)

In [ ]:
# Number of Work Order Templates

url = base_url + 'Ams/ServiceRequestTemplate/Search'
data = {"IsActive": 1}
parameters = data_to_json(data)
results = make_request(url, parameters)

print("Number of Service Request Templates from DB SQL: {}".format(str(srtemplates)))
print("Number of Service Request Templates from CW API: {}\n".format(str(len(results['Value']))))

url = base_url + 'Ams/InspectionTemplate/Search'
data = {"IsActive": 1}
parameters = data_to_json(data)
results = make_request(url, parameters)

print("Number of Inspection Templates from DB SQL: {}".format(str(insptemplates)))
print("Number of Inspection Templates from CW API: {}\n".format(str(len(results['Value']))))

url = base_url + 'Ams/WorkOrderTemplate/Search'
data = {"IsActive": 1}
parameters = data_to_json(data)
results = make_request(url, parameters)

print("Number of Work Order Templates from DB SQL: {}".format(str(wotemplates)))
print("Number of Work Order Templates from CW API: {}\n".format(str(len(results['Value']))))

In [ ]:
# 1. Create CW Inspection

insp_template = 70 # Created an DOS Asset Photo Inspection Template
entity_type = 'GUARDRAILS'

url = base_url + 'Ams/Inspection/Create'
data = {"EntityType": entity_type, "InspTemplateId": insp_template}
parameters = data_to_json(data)
insp_create = make_request(url, parameters)

insp_id = insp_create['Value']['InspectionId']

# print(insp_create['Value'])

In [ ]:
print("InspectionID: {}".format(insp_create['Value']['InspectionId']))
inspection_url = r'https://www.cityworksurl.com/cityworks/Workmanagement/InspectionEdit.aspx?InspectionId=' + str(insp_create['Value']['InspectionId'])
webbrowser.open(inspection_url)

In [ ]:
# 2. Update CW inspection

url = base_url + 'Ams/Inspection/Update'

data = {"InspectionId": insp_id,
        "DateSubmitTo": '0001-01-01 00:00:00',
        "InspectionDate": '2021-04-15 09:00:00', # today's date
        "InspectedBy": str(12698), # inspector UID
        }

parameters = data_to_json(data)
insp_update = make_request(url, parameters)
print('Status: {}'.format(str(insp_update['Status'])))

In [ ]:
# 3. Add entity to connect to GIS

url = base_url + 'Ams/Inspection/AddEntity'

data = {
    "EntityType": entity_type,
    "InspectionId": insp_id,
    "EntityUid": '53254'
    }
parameters = data_to_json(data)
entity_add = make_request(url, parameters)

print('Status: {}'.format(str(entity_add['Status'])))

In [ ]:
# 4. Attach a list of photos

filepath = r'C:\data\Cityworks_Innovate_2021\pics'
photo_list = [os.path.join(filepath, 'guardrail_photo.jpg'), os.path.join(filepath, 'jimi-hendrix.jpg')]

for filepath in photo_list:
    attach = open(filepath, "rb")
    attaches = {"file": (os.path.basename(filepath), attach)}
    data = {
        "InspectionId": insp_id,
    }

    url = base_url + 'Ams/Attachments/AddInspectionAttachment'
    parameters = data_to_json(data)
    response = requests.post(url=url, files=attaches, data=parameters)
    attach_response = json.loads(response.text)
    print('Status: {}'.format(str(attach_response['Status'])))

In [ ]:
# 5. Close Inspection

url = base_url + 'Ams/Inspection/Close'
data = {"InspectionIds": [insp_id]}
parameters = data_to_json(data)
insp_close = make_request(url, parameters)
print('Status: {}'.format(str(insp_close['Status'])))

In [ ]:
# Read a table (csv or excel)
# Separated into two tables
# One table has just the assets
# Second Table has the photos connected to the assets because you can have many photos to one asset

xlsx_file = r'C:\data\Cityworks_Innovate_2021\Guardrails_Asset_only.xlsx'
df = pandas.read_excel(io=xlsx_file, sheet_name='Sheet1')

xlsx_file = r'C:\data\Cityworks_Innovate_2021\Guardrails_Asset_Photo_ID.xlsx'
df_photo = pandas.read_excel(io=xlsx_file, sheet_name='Sheet1')

In [ ]:
df

In [ ]:
df_photo

In [ ]:
# All together iterating through the rows of a table
# to create, update, connect to GIS entity, upload photos, and close an inspection

for i in df.index:
    if not pandas.isna(df['assetid'][i]): # if there is no asset ID, skip it
        print("Asset ID: {}".format(str(df['objectid'][i])))

        # 1. Create an Inspection
        url = base_url + 'Ams/Inspection/Create'
        data = {"EntityType": entity_type, "InspTemplateId": insp_template}
        parameters = data_to_json(data)
        insp_create = make_request(url, parameters)

        insp_id = insp_create['Value']['InspectionId']

        print("Inspection ID: {}".format(str(insp_id)))

        # 2. Update inspection
        url = base_url + 'Ams/Inspection/Update'

        data = {"InspectionId": insp_id,
                "DateSubmitTo": '0001-01-01 00:00:00',
                "InspectionDate": str(df['AssetCollectionDate'][i]),
                "InspectedBy": str(12698),
                }

        parameters = data_to_json(data)
        insp_update = make_request(url, parameters)

        print("Inspection Updates: {}".format(str(insp_update['Status'])))

        # 3. Connect to GIS entity
        url = base_url + 'Ams/Inspection/AddEntity'
        data = {
            "EntityType": entity_type,
            "InspectionId": insp_id,
            "EntityUid": str(df['assetid'][i])
            }
        parameters = data_to_json(data)
        entity_add = make_request(url, parameters)

        print("Entity Add Results: {}".format(str(entity_add['Status'])))

        # 4. Attach a list of photos
        photo_list = []
        objectid = str(df['assetid'][i])
        rows = df_photo.index[df_photo['sdeglobalid'].isin([df['sdeglobalid'][i]])].tolist()
        if len(rows) >= 1:
            for row in rows:
                photo_list.append(df_photo['photoloc'][row])

        for filepath in photo_list:
            attach = open(filepath, "rb")
            attaches = {"file": (os.path.basename(filepath), attach)}
            data = {
                    "InspectionId": insp_id,
                    }

            url = base_url + 'Ams/Attachments/AddInspectionAttachment'
            parameters = data_to_json(data)
            response = requests.post(url=url, files=attaches, data=parameters)
            attach_response = json.loads(response.text)

            print("Photo Attach: {}".format(str(attach_response['Status'])))


        # 5. Close inspection
        url = base_url + 'Ams/Inspection/Close'
        data = {"InspectionIds": [insp_id]}

        parameters = data_to_json(data)
        insp_close = make_request(url, parameters)

        print("Inspection Closed: {}".format(str(insp_close['Status'])))

        print("Complete Insp: {}" .format(str(insp_id)))

        inspection_url = r'https://www.cityworksurl.com/cityworks/Workmanagement/InspectionEdit.aspx?InspectionId=' + str(insp_id)
        print(inspection_url)
        print("\n")
        #webbrowser.open(inspection_url)

In [ ]:
# Final Thoughts on Simplifying Data Management Using the Cityworks API and Python
# 
# 1. Can you do the process manually?
# 2. Will you need to rerun this multiple times?
# 3. Script is supposed to save time
#    - developing a fully robust, error proof, user proof script may not be what is exactly needed
# 4. Be familiar with the data
# 5. Technology is always improving
# 6. Always develop and test on a non-production Cityworks instance

In [ ]:
# Moving forward with Cityworks at Cleveland
#
# 1. Upgrade to 15.6.4
# 2. Another domain
# 3. Another implementation into an existing domain
# 4. Automate quality assurance testing for future upgrades and enhancements (more Python!)
# 5. Postman for additional assistance and increasing API efficiency

In [ ]:
# github link with Jupyter notebook

github.com/milkor56/conferences

In [ ]:
# Contact Information
#
# Miklos Nadas, GISP
# City of Cleveland
# miklos_nadas@clevelandwater.com
#